# Pandas and Visualization

pandas is a library that is indended to make data analysis easier.

The central concept in pandas is a "dataframe". These dataframes are similar (in many respects) to Excel spreadsheets. They are typically 2-dimensional representations of data.

The first thing that we need to do is to import pandas. By convention, we rename it to "pd".

We're also importing another visualization library called seaborn. Again, by convention it's imported as "sns"

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Getting the data

For now, we are assuming that you have data from somewhere else, in a [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values). For today's exercises, we're going to use data that I got from [BigQuery](https://console.cloud.google.com/bigquery), which is a tool produced by Google for querying huge datasets.

The folks at [Pushshift](https://pushshift.io/) gather and publish a bunch of the data from reddit. I queried their data to get a random set of 100K comments from 2018. You can download that data [here](https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/raw/master/resources/data/100k_random_reddit_2018.csv).

First, we load it into a dataframe. Change the path to where you are storing your data.

In [ ]:
reddit_df = pd.read_csv('/home/jeremy/Teaching/intro_to_programming_and_ds/resources/data/100k_random_reddit_2018.csv')

The "head" method lets us look at the first few rows of the data

In [ ]:
reddit_df.head()

As you can see, this is organized a lot like a spreadsheet or a CSV file. Each row is an observation, and each column is a measure about that observation. In this case, a row represents a comment on reddit.

We are going to be talking a lot more about how to manipulate and work with this data. This document is intended to introduce a few key ideas.

## Selecting data

The first is selecting data - getting a subset of the data.

This is a really confusing thing about pandas and takes some getting used to. There is a detailed explanation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) but I'm going to try to give a gentler introduction here.

The main thing that trips people up is remembering that dataframes have an "index". By default, this is a number that refers to each row in the dataframe. If you look at our dataframe, you will see that on the far left is an unnamed column of numbers starting at 0. This is the index.

It is assigned in the order that the rows were in the CSV file. However, it can change if we reorder things.

For example, this code will sort the dataframe by comment length. Notice how the index numbers are now out of order.

In [ ]:
df_by_length = reddit_df.sort_values('comment_length', ascending=False)
df_by_length.head()

We can use this "out of order" version of the data fram to show the difference between the two main ways of selecting data: `iloc` and `loc`.

`iloc` selects data like a list index, based on the location in the current dataframe. The syntax is just like selecting elements from two lists: the row list and the column list. If we use slice notation it looks like this:

`dataframe_name.iloc[starting_row:ending_row, staring_column:ending_column]`

You can also select a list of row numbers or column numbers. Below, I pass a list of columns to select.

In [ ]:
# This code gets the first 10 rows and columns 1, 3, and 4
df_by_length.iloc[0:10, [1,3,4]]

The `loc` syntax, on the other hand, selects based on labels. It selects rows based on index labels and columns based on column names. When something is sorted, this can have unexpected consequences. 

For example, we might think that the code below would just select the first three rows, or maybe the three rows with indexes 0,1, and 2. Instead, it starts with the item with index `0`, and selects all of the rows from that row until the row with index `3`, as currently sorted! Notice at the bottom of the output, it says that the length is 1,494!

In [ ]:
df_by_length.loc[0:3, 'comment_length']

## Boolean indexing

Typically, if we want to get a certain number of rows, we use `.iloc`. However, if we want to filter data based on its value, then we typically use `.loc`

For example, if we wanted to get the first 3 rows by their index, we'd need to do something like this, which uses "boolean indexing" to filter to only the rows where the index is less than or equal to 3.

In [ ]:
df_by_length.loc[df_by_length.index < 3,"comment_length":]

Or, if we wanted to just get the subreddit and the score whenever scores are above the median score, we could do this:

In [ ]:
df_by_length.loc[df_by_length.score > df_by_length.score.median(),['subreddit', 'score']]

Finally, it is possible to filter dataframes without using `iloc` or `loc`

One that's useful is dot notation - as long as your column names don't have spaces, you can do something like:

In [ ]:
# Just gives one column
df_by_length.score

# This is equivalent, and can handle spaces
df_by_length['score']

In [ ]:
# If you use slicing notation, this will give you rows (like iloc)

df_by_length[:5]

### Exercise 1

Sort the data frame by score and select the rows with the 10th to 20th highest scores.


In [ ]:
## Your code here

### Exercise 2

Now, how would you select just the `subreddit` and `comment_length` columns for the rows you selected above?

In [ ]:
## Your code here

## Grouping and aggregating

Pandas is also great for grouping and aggregating data.

We're going to talk a lot more about analyzing data but here's a quick introduciton to some great built in functions like max, sum, and mean

In [ ]:
# Get the mean score and comment length

df_by_length.loc[:,"comment_length":"score"].mean()

We also might want to create subreddit-level or person-level measures. To do this, we'll want to "group" the data.

The groupby function will create a "grouped" dataframe, where aggregations apply to the group rather than the whole dataframe. I think of this as temporarily sorting all of the rows into their own mini-spreadsheets based on whatever is in the `groupby` function.

For example, this will tell use the average comment score by subreddit:

In [ ]:
# Note that I'm using the original dataframe but either one would work

# Groups by subreddit, then gets the score column
# Then gets the mean of that column, then sorts it by the value of the column
reddit_df.groupby('subreddit').score.median().sort_values() 

As another example, this will get the number of comments per person:

In [ ]:
reddit_df.groupby('author').size().sort_values()

### Exercise 3

Find the average (mean) comment length for each subreddit and sort by comment length

In [ ]:
## Your code here

We can talk more about this in class, but this creates a user-based dataframe with data for the number of subreddits, number of comments, and average score for each user.

In [ ]:
grouped_df = reddit_df.groupby('author')

subreddit_count = grouped_df.subreddit.nunique()
comments_count = grouped_df.size()
score_mean = grouped_df.score.mean()


person_df = pd.DataFrame({'subreddit_count': subreddit_count,
                          'comments_count': comments_count,
                          'mean_score': score_mean})

In [ ]:
person_df.head()

# Visualizations

Python has a powerful (and complicated) plotting library called matplotlib. If you want to do more complicated plots, I highly recommend learning how to use it. For now, I'm going to introduce the visualizations that are part of pandas, and later introduce some which are part of Seaborn. Both of these are based on matplotlib but make it easier and simpler to use.

I'm introducing a few basic visualizations here. You should [read and reference this page](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) to learn about a bunch more options.

### Scatterplots

I think it would be interesting to know if there's a relationship between the length of comments and the score. This plot shows the scatterplot of that relationship.

In [ ]:
reddit_df.plot.scatter(x = 'comment_length', y = 'score');

Both of these measures are really skewed, so let's try plotting them on log scales instead.

In [ ]:
## Note that this is slightly different (improved) from the code in the video
reddit_df['comment_logged'] = np.log1p(reddit_df.comment_length)
# Scores can be negative, so we add the minimum (plus one) to make sure they are positive
# And then log transform
reddit_df['score_logged'] = np.log1p(reddit_df.score + abs(reddit_df.score.min()))

In [ ]:
reddit_df.plot.scatter(x = 'comment_logged', y = 'score_logged');

Not much of a relationship there.

### Exercise 4

Plot the number of comments and average score per person. Are frequent posters more likely to share comments that are well-received?

*Hint*: You will want to use a dataframe that groups the data by person

In [ ]:
## Your code here



## Time Series

The last thing I want to show is how to do time series (because they are confusing!)

They key thing to remember is that the plots assume that time is the index of the data frame. By default, the index is just the row number of the row when it was first imported.

So, the first thing we need to do is to convert the datetime column to the index.

`reddit_df.index` is where the index is stored, so the following code takes the `created_utc` column--which is the [Unix time](https://en.wikipedia.org/wiki/Unix_time) that each comment was posted--and converts it to a "DateTime", which is a way of representing a date and time in Python. It then stores the datetime in the index.

In [ ]:
reddit_df.index = pd.to_datetime(reddit_df.created_utc, unit='s', utc=True)
reddit_df.head()

In order to make a time series, we have to combine the data. The `resample` function does this. Below, we combine it by hour to see daily trends.

In [ ]:
hourly_comments = reddit_df.resample('H').size()

In [ ]:
hourly_comments

In [ ]:
hourly_comments.plot();

This is too messy, so let's zoom in on just a few weeks, using boolean indexing

In [ ]:
hourly_comments.loc[hourly_comments.index < '2018-01-18'].plot()

### Exercise 5

Plot the number of comments per month over the year.

In [ ]:
# Your code here

Finally, if we wanted to summarize by hour, we could group the posting time by hour. Below, we see that about 8:00 UTC (which is about 3:00 AM Eastern) corresponds to the lowest activity time.

In [ ]:
reddit_df.groupby(reddit_df.index.hour).size().plot();

### Exercise 6

See if you can figure out how to change the above plot so that instead of showing one overall line for comments per hour, it shows a line for each day of the week (e.g., a line for comments per hour on Mondays, another for Tuesdays, etc.).

This code can help you to get started: it sums the number of comments by hour and day.

In [ ]:
reddit_df.groupby([reddit_df.index.hour, reddit_df.index.weekday]).size()